In [1]:
%reset -f 
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import UnivariateSpline
import time
import derivatives as d
from forward_sim import Chameleon

In [2]:
chameleon = Chameleon(E=10)

In [3]:
active_stress = 1/10*np.linspace(1, 10, chameleon.n_elems)

In [4]:
active_stress

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

## This is too slow. In order to do RL I want to be able to simulate much faster than this. (Jax?)

### could also simulate for a shorter time when I want to.

I could almost already build this as an RL environment. Just need to wrap this around a gym environment, reset the thing according to the most recent position and have each specification of torque account for a smaller period of simulation time. Maybe do 1000 steps of sim instead of 10000.


In [5]:
%%time
chameleon.step(active_stress)

CPU times: user 812 µs, sys: 170 µs, total: 982 µs
Wall time: 858 µs


## Why this error when I increase young's modulus? That seems weird

In [6]:
%%time
chameleon.forward_simulate(active_stress, sim_steps=1000)

CPU times: user 210 ms, sys: 3.23 ms, total: 213 ms
Wall time: 212 ms


In [7]:
chameleon.pos_f

array([0.        , 0.2083041 , 0.40772997, 0.60570449, 0.80224343,
       0.99736232, 1.19107649, 1.38340104, 1.57435084, 1.76394057])

In [8]:
chameleon.disp_current

array([0.        , 0.09719299, 0.18550775, 0.27237116, 0.35779899,
       0.44180677, 0.52440983, 0.60562326, 0.68546195, 0.76394057])

In [9]:
len(chameleon.position_history)

1000

# I would like to see some kind of visualization of the position history here